In [ ]:
# Setup Folders, Tokens, and Dependencies
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import plotly.express as px
from pathlib import Path

datasets_folder = Path('./data')
px.set_mapbox_access_token(open(".mapbox_token").read())

# Loading the Data

In [ ]:
temp_gdf = gpd.read_file(datasets_folder / 'temperature.geojson')
bio_land_gdf =  gpd.read_file(datasets_folder / 'biodiversity_land.geojson')
bio_sea_gdf =  gpd.read_file(datasets_folder / 'biodiversity_sea.geojson')
disaster_gdf = gpd.read_file(datasets_folder / 'disaster.geojson')

**APP IDEAS**
- The initial page will an animated of a short text on the center that catches the viewers' attention. It can be a call to action or a hook to incite curiosity.
- Below the Text will be a button to "Click to Learn More" or "Continue", leading to the Temperature page
- All pages will contain a header/nav that contains an offcanvas for easier navigation between pages

# Temperature

In [ ]:
print(temp_gdf.shape)
temp_gdf.head()

### TEMP 1: The first visualization for the temperature data is an **Animated Bar Graph** that shows the Average Temperature per Province in [Island Group] across all of the decades from 1960s to 2020s.
#### Interactions:
- Across the Decades Animation
- DropDown/RadioButtons that switch [Island Group] "Luzon, Visayas, and Mindanao" through Dash

In [ ]:
# Sliced GDF by Island Group
luzon_gdf = temp_gdf[(temp_gdf['island_group'].isin(['Visayas']) == True)] # Change Island Group based on Dropdown Value

# Create the Figure with horizontal orientation
fig = px.bar(luzon_gdf, y='name', x='value', animation_frame="decade", orientation='h')

# Update the layout
fig.update_layout(
    autosize=True,
    height=600,  # Set total height to accommodate the graph and controls
    title='Average Temperature per Province in Luzon',
    margin=dict(l=20, r=20, t=50, b=50),
)

# Update y-axis label
fig.update_yaxes(title_text="Province Name")  

# Update x-axis label
fig.update_xaxes(title_text="Temperature (°C)", 
                 range=[0, 33],  # Set the range for the x-axis
                 tickmode='linear', 
                 dtick=2,  # Set the tick interval for the x-axis
                 tickangle=0)  # Set the angle of the x-axis labels)  

# Update hover template
hover_template = "Average Temperature in<br>" + \
                 "<b>%{y}</b><br>" + \
                 "during the %{customdata[0]}:<br>" + \
                 "%{customdata[1]:.2f}°C"

fig.update_traces(hovertemplate=hover_template,
                  customdata=luzon_gdf[['decade', 'value']])

for frame in fig.frames:
    frame.data[0].hovertemplate = hover_template
    frame.data[0].customdata = luzon_gdf[['decade', 'value']]

# Update menus and sliders position
fig.update_layout(
    updatemenus=[{
        'direction': 'left',  
        'pad': {'t': 10, 'b': 10, 'l': 10, 'r': 10},  # Adjust padding as needed
        'showactive': False,
        'type': 'buttons',
        'x': 0.06,  
        'xanchor': 'right',
        'y': -0.14,  # Position the menu below the graph
        'yanchor': 'top'
    }],
    sliders=[{
        'active': 0,
        'x': 0.98,   
        'y': -0.08,  # Position the slider below the graph
        'xanchor': 'right',
        'yanchor': 'top',
        'transition': {'duration': 300, 'easing': 'cubic-in-out'},
        'pad': {'t': 10, 'b': 10, 'l': 10, 'r': 10},  # Adjust padding as needed
        'currentvalue': {
            'font': {'size': 16},
            'prefix': 'Decade:',
            'visible': True,
            'xanchor': 'right',
        },
        'visible': True,
    }]
)

fig.show()


### TEMP 2:  The second visualization for the temperature data is a **Chloropleth Map** that shows the Average Temperature per Province across all of the decades from 1960s to 2020s.
#### Interactions:
- Across the Decades Animation

<em>Note: the reason that only four decades are used (1960s, 1980s, 2000s, and 2020s) is because using more decades/dimensions would make the load time of the figure significantly higher reaching up to an hour. This visualization takes approximately 2-3 minutes to load in this notebook. </em>

In [ ]:
# Initialize the years
temp_sliced = temp_gdf[(temp_gdf['decade'].isin(['1960s', '1980s', '2000s', '2020s']) == True)].reset_index()

# Create the Figure
fig = px.choropleth_mapbox(temp_sliced,
                           geojson=temp_sliced.geometry,
                           locations=temp_sliced.index,
                           color='value',
                           color_continuous_scale='turbo',
                           range_color=[25, 32.5], 
                           mapbox_style='streets',
                           zoom=5, 
                           center={"lat": 12.8797, "lon": 121.7740},
                           opacity=0.6,
                           animation_frame="decade"
                           )

fig.update_layout(coloraxis_colorbar=dict(title="Average<br>Temperature (°C)",yanchor="top",xanchor='right',
                            y=1, x=1, ticks="outside", thickness=10, title_font_color='#0c232c',
                            tickvals=[i for i in range(25, 33)],  
                            tickmode='array',  
                            ticksuffix='°C',
                            tickfont=dict(
                            size=12,
                            color='#0c232c')
                        )
                    )

fig.update_layout(margin=dict(l=0, r=0, t=0, b=0),
    updatemenus=[{
        'buttons': [
                {'visible': True}
        ],
        'direction': 'left',  
        'pad': {'t': 0, 'b': 0, 'l': 0, 'r': 0},  
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,  
        'xanchor': 'right',
        'y': 0.24,  
        'yanchor': 'top'
    }],
    sliders=[{
        'active': 0,
        'x': 0.05,   
        'y': 0.19, 
        'len': 0.5,
        'xanchor': 'left',
        'yanchor': 'top',
        'currentvalue': {
            'font': {'size': 16},
            'prefix': 'Decade:',
            'visible': True,
            'xanchor': 'right'
        },
        'transition': {'duration': 1000, 'easing': 'cubic-in-out'},
        'pad': {'t': 0, 'b': 0, 'l': 0, 'r': 0}
    }]
)

hover_template = '<b>%{customdata[0]}</b><br>Average Temp: %{customdata[1]:.2f}°C<extra></extra>'

fig.update_traces(hovertemplate=hover_template,
                  customdata=temp_sliced[['name', 'value']])

for frame in fig.frames:
    frame.data[0].hovertemplate = hover_template
    frame.data[0].customdata = temp_sliced[['name', 'value']]

fig.show()

**APP IDEAS**
- The temperature page will be in two columns 3:9
- The right one is the map TEMP 2
- The left one is a header and story explaining the map
- below the text will be a modal button "See More" that opens a modal
- The modal contains two columns, the left one is a text header and story, the right one is TEMP 1 bar graph
- The app will have a footer that has two buttons at its ends, "< BACK" and "NEXT >"
- Back will lead to the initial page and Next will lead to the Biodiversity Page

# Biodiversity

In [ ]:
print(bio_land_gdf.shape)
bio_land_gdf.head()

### BIO 1: The first visualization for the biodiversity data is a **Chloropleth Map** that shows the number of [Total/CR/EN/VU] unique species per Province.
#### Interactions:
- DropDown/RadioButtons that switch between "Total, CR, EN and VU" through Dash

In [ ]:
fig = px.choropleth_mapbox(bio_land_gdf,
                            geojson=bio_land_gdf.geometry,
                            locations=bio_land_gdf.index,
                            color='Critically Endangered', # Change this based on dropdown
                            color_continuous_scale='balance',
                            range_color=[bio_land_gdf['Critically Endangered'].min(), bio_land_gdf['Critically Endangered'].max()],  
                            mapbox_style='streets',  
                            zoom=5,  
                            center={"lat": bio_land_gdf.geometry.centroid.y.mean(), 
                                    "lon": bio_land_gdf.geometry.centroid.x.mean()}, 
                            opacity=0.6,
                            )

fig.update_layout(margin=dict(l=0, r=0, t=0, b=0))

fig.update_layout(coloraxis_colorbar=dict(title="Critically Endangered<br>Species",yanchor="top",xanchor='right',
                            y=1, x=1, ticks="outside", thickness=10, title_font_color='#0c232c',  
                            ticksuffix=' Species',
                            tickfont=dict(
                            size=12,
                            color='#0c232c')
                        )
                    )

hover_template = '<b>%{customdata[0]}</b><br>Unique Species: %{customdata[1]:.0f}<extra></extra>'

fig.update_traces(hovertemplate=hover_template,
                  customdata=bio_land_gdf[['name', 'Critically Endangered']])

fig.show()

### BIO 2: The second visualization for the biodiversity data is a DRILLDOWN operation on the previous **Chloropleth Map** that shows the number of [Total/CR/EN/VU] unique species per Province. In addition to this, the drilldown will be based on Island Groups
#### Interactions:
- A button that triggers the drilldown
- DropDown/RadioButtons that switch between "Total, CR, EN and VU" (from the previous map BIO 1)
- DropDown/RadioButtons that switch between "Luzon, Visayas, and Mindanao"

In [ ]:
# Set Island Group
curr_group = bio_land_gdf[bio_land_gdf['island_group'] == 'Luzon']

# Createe the figure
fig = px.choropleth_mapbox(curr_group,
                            geojson=curr_group.geometry,
                            locations=curr_group.index,
                            color='Critically Endangered', # Change this based on dropdown
                            color_continuous_scale='balance',
                            range_color=[curr_group['Critically Endangered'].min(), curr_group['Critically Endangered'].max()],  
                            mapbox_style='streets',  
                            zoom=6,  
                            center={"lat": curr_group.geometry.centroid.y.mean(), 
                                    "lon": curr_group.geometry.centroid.x.mean()}, 
                            opacity=0.6,
                            )

fig.update_layout(margin=dict(l=0, r=0, t=0, b=0))

fig.update_layout(coloraxis_colorbar=dict(title="Critically Endangered<br>Species",yanchor="top",xanchor='right',
                            y=1, x=1, ticks="outside", thickness=10, title_font_color='#0c232c',  
                            ticksuffix=' Species',
                            tickfont=dict(
                            size=12,
                            color='#0c232c')
                        )
                    )

hover_template = '<b>%{customdata[0]}</b><br>Unique Species: %{customdata[1]:.0f}<extra></extra>'

fig.update_traces(hovertemplate=hover_template,
                  customdata=bio_land_gdf[['name', 'Critically Endangered']])

fig.show()

### BIO 3: The third visualization for the biodiversity data is ALSO a DRILLDOWN operation on the first chloropleth map and is a **Horizontal Stacked Bar Chart** that shows the number of [CR/EN/VU] unique species per Province. In addition to this, the drilldown will be based on Island Groups
#### Interactions:
- A button that triggers the drilldown (same button as BIO 2)
- DropDown/RadioButtons that switch between "Luzon, Visayas, and Mindanao" (same button as BIO 2)

In [ ]:
# Island Group
luzon_data = bio_land_gdf[bio_land_gdf['island_group'] == 'Luzon'].sort_values(by='total_species', ascending=True)

# Create stacked bar plot using Plotly Express
fig = px.bar(luzon_data,
             x=['Vulnerable', 'Endangered', 'Critically Endangered'],  
             y='name',  
             title='Number of Unique Endangered Species in Luzon',
             labels={'Vulnerable', 'Endangered', 'Critically Endangered'},  
             orientation='h',  
             height=600,  
             color_discrete_sequence=['yellow', 'orange', 'red'], 
             )

fig.update_layout(
    xaxis_title='Number of Unique Species', 
    yaxis_title='Province',
    legend_title='Category',
)

hover_template = '<b>%{customdata[0]}</b><br>Unique Species: %{x}<extra></extra>'

fig.update_traces(hovertemplate=hover_template,
                  customdata=luzon_data[['name']])

fig.show()

In [ ]:
print(bio_sea_gdf.shape)
bio_sea_gdf.head()

### BIO 4: The fourth visualization for the biodiversity data is a **Chloropleth Map** that shows the number of [Total/CR/EN/VU] unique species per sea.
#### Interactions:
- DropDown/RadioButtons that switch between "Total, CR, EN and VU" through Dash

In [ ]:
fig = px.choropleth_mapbox(bio_sea_gdf,
                            geojson=bio_sea_gdf.geometry,
                            locations=bio_sea_gdf.index,
                            color='Critically Endangered', # Change this based on dropdown
                            color_continuous_scale='sunsetdark',
                            range_color=[bio_sea_gdf['Critically Endangered'].min(), bio_sea_gdf['Critically Endangered'].max()],  
                            mapbox_style='streets',  
                            zoom=5,  
                            center={"lat": bio_sea_gdf.geometry.centroid.y.mean(), 
                                    "lon": bio_sea_gdf.geometry.centroid.x.mean()}, 
                            opacity=0.6,
                            )

fig.update_layout(margin=dict(l=0, r=0, t=0, b=0))

fig.update_layout(coloraxis_colorbar=dict(title="Critically Endangered<br>Species",yanchor="top",xanchor='right',
                            y=1, x=1, ticks="outside", thickness=10, title_font_color='#0c232c',  
                            ticksuffix=' Species',
                            tickfont=dict(
                            size=12,
                            color='#0c232c')
                        )
                    )

hover_template = '<b>%{customdata[0]}</b><br>Unique Species: %{customdata[1]:.0f}<extra></extra>'

fig.update_traces(hovertemplate=hover_template,
                  customdata=bio_sea_gdf[['name', 'Critically Endangered']])

fig.show()

### BIO 5: The fifth visualization for the biodiversity data is a **Horizontal Stacked Bar Chart** that shows the number of [CR/EN/VU] unique species per Sea.
#### Interactions:
- A button that shows the visualization

In [ ]:
# Create stacked bar plot using Plotly Express
fig = px.bar(bio_sea_gdf,
             x=['Vulnerable', 'Endangered', 'Critically Endangered'],  
             y='name',  
             title='Number of Unique Endangered Species in Sea',
             labels={'Vulnerable', 'Endangered', 'Critically Endangered'},  
             orientation='h',  
             height=600,  
             color_discrete_sequence=['yellow', 'orange', 'red'], 
             )

fig.update_layout(
    xaxis_title='Number of Unique Species', 
    yaxis_title='Province',
    legend_title='Category',
)

hover_template = '<b>%{customdata[0]}</b><br>Unique Species: %{x}<extra></extra>'

fig.update_traces(hovertemplate=hover_template,
                  customdata=bio_sea_gdf[['name']])

fig.show()

**APP IDEAS**
- There is toggle that switches the page to show Land Data or Sea Data
- Bio 1, Bio 2, and Bio 3 is part of the Land and will have interactions that are connected to each other
- Land data will somehow be related to the temp data
- Bio 4, and Bio 5 is part of the Sea and will havee interactions that are also connected to each other
- Will also be compared to temp data

# Disaster

In [ ]:
print(disaster_gdf.shape)
disaster_gdf.head()

### DISASTER 1: The first visualization for the disaster data is a **Chloropleth Map** that shows the number of [Disaster Type] per Province.
#### Interactions:
- DropDown/RadioButtons that switch between Disaster Types (Total, Storm, Flood, Earthquake, Volcanic, Mass Movement, Drought) through Dash

In [ ]:
fig = px.choropleth_mapbox(disaster_gdf,
                            geojson=disaster_gdf.geometry,
                            locations=disaster_gdf.index,
                            color='Total Disaster Count', # Change based on dropdown value
                            color_continuous_scale='reds',
                            range_color=[disaster_gdf['Total Disaster Count'].min(), disaster_gdf['Total Disaster Count'].max()],  
                            mapbox_style='streets',  
                            zoom=5,  
                            center={"lat": disaster_gdf.geometry.centroid.y.mean(), 
                                    "lon": disaster_gdf.geometry.centroid.x.mean()}, 
                            opacity=0.6,
                            )

fig.update_layout(margin=dict(l=0, r=0, t=0, b=0))

fig.update_layout(coloraxis_colorbar=dict(title="Total Disaster<br>Count",yanchor="top",xanchor='right',
                            y=1, x=1, ticks="outside", thickness=10, title_font_color='#0c232c',  
                            # ticksuffix=' disasters',
                            tickfont=dict(
                            size=12,
                            color='#0c232c')
                        )
                    )

hover_template = '<b>%{customdata[0]}</b><br>%{customdata[1]:.0f} disasters<extra></extra>'

fig.update_traces(hovertemplate=hover_template,
                  customdata=disaster_gdf[['Area Name', 'Total Disaster Count']])

fig.show()

### DISASTER 2: The second visualization for the disaster data is a **Horizontal Bar Chart** that shows the number of [Disaster Type] per Province per [Island Group].
#### Interactions:
- DropDown/RadioButtons that switch between Disaster Types (Total, Storm, Flood, Earthquake, Volcanic, Mass Movement, Drought) through Dash (Same button as Disaster 1 visualization)
- DropDown/RadioButtons that switch between Island Groups (Luzon Visayas Mindanao)

In [ ]:
# Island Group
luzon_disaster = disaster_gdf[disaster_gdf['Island Group'] == 'Luzon'].sort_values(by='Total Disaster Count', ascending=True)

# Create stacked bar plot using Plotly Express
fig = px.bar(luzon_disaster,
             x='Total Disaster Count',  
             y='Area Name',  
             title='Total Disasters in Luzon per Province',
             orientation='h',  
             height=600,  
             color_discrete_sequence=['lightblue'], 
             )

fig.update_layout(
    xaxis_title='Total Disasters', 
    yaxis_title='Province',
)

hover_template = '<b>%{customdata[0]}</b><br>%{x} disasters<extra></extra>'

fig.update_traces(hovertemplate=hover_template,
                  customdata=luzon_disaster[['Area Name']])

fig.show()

### DISASTER 3: The third visualization for the disaster data is a **Chloropleth Map** that shows the number of [Disaster Type] per REGION.
#### Interactions:
- DropDown/RadioButtons that switch between Disaster Types (Total, Storm, Flood, Earthquake, Volcanic, Mass Movement, Drought) through Dash

In [ ]:
Region_gdf = disaster_gdf.copy()
Region_tot_ave = Region_gdf.drop(columns=['geometry']).groupby('Region').sum()

def region_count(disaster_type,col_name):
    Region_gdf[col_name] = np.nan #Create a new column that contains the number of disasters per region

    for i1,r1 in Region_gdf.iterrows():
        for i2,r2 in Region_tot_ave.iterrows():
            if r1['Region'] == i2:
                Region_gdf.at[i1, col_name] = r2[disaster_type]
            else:
                continue

# Total Disasters per Region
region_count('Total Disaster Count','Region_tot')
# Storm per Region
region_count('Storm Count','Region_storm')
# Flood per Region
region_count('Flood Count','Region_flood')
# Earthquake per Region
region_count('Earthquake Count','Region_earth')
# Volcanic Activity per Region
region_count('Volcanic Activity Count','Region_vol')
# Mass Movement per Region
region_count('Mass Movement Count','Region_mass')
# Drought per Region
region_count('Drought Count','Region_drought')
Region_gdf.head()

In [ ]:
fig = px.choropleth_mapbox(Region_gdf,
                            geojson=Region_gdf.geometry,
                            locations=Region_gdf.index,
                            color='Region_tot', # Change based on dropdown value
                            color_continuous_scale='amp',
                            range_color=[Region_gdf['Region_tot'].min(), Region_gdf['Region_tot'].max()],  
                            mapbox_style='streets',  
                            zoom=5,  
                            center={"lat": Region_gdf.geometry.centroid.y.mean(), 
                                    "lon": Region_gdf.geometry.centroid.x.mean()}, 
                            opacity=0.6,
                            )

fig.update_layout(margin=dict(l=0, r=0, t=0, b=0))

fig.update_layout(coloraxis_colorbar=dict(title="Total Disaster<br>Count",yanchor="top",xanchor='right',
                            y=1, x=1, ticks="outside", thickness=10, title_font_color='#0c232c',  
                            # ticksuffix=' disasters',
                            tickfont=dict(
                            size=12,
                            color='#0c232c')
                        )
                    )

hover_template = '<b>%{customdata[0]}</b><br>%{customdata[1]:.0f} disasters<extra></extra>'

fig.update_traces(hovertemplate=hover_template,
                  customdata=Region_gdf[['Region', 'Region_tot']])

fig.show()

### DISASTER 4: The fourth visualization for the disaster data is a **Horizontal Bar Chart** that shows the number of [Disaster Type] per REGION per [Island Group].
#### Interactions:
- DropDown/RadioButtons that switch between Disaster Types (Total, Storm, Flood, Earthquake, Volcanic, Mass Movement, Drought) through Dash (Same button as Disaster 3 visualization)
- DropDown/RadioButtons that switch between Island Groups (Luzon Visayas Mindanao)

In [ ]:
# Island Group
luzon_reg_disaster = Region_gdf[Region_gdf['Island Group'] == 'Luzon'].groupby('Region')['Region_tot'].sum().sort_values(ascending=True)

# Create stacked bar plot using Plotly Express
fig = px.bar(x=luzon_reg_disaster.values,  
             y=luzon_reg_disaster.index,  
             title='Total Disasters in Luzon per Region',
             orientation='h',  
             height=600,  
             color_discrete_sequence=['blue'], 
             )

fig.update_layout(
    xaxis_title='Total Disasters', 
    yaxis_title='Region',
)

hover_template = '<b>%{y}</b><br>%{x} disasters<extra></extra>'

fig.update_traces(hovertemplate=hover_template,
                  customdata=luzon_reg_disaster.reset_index().values)

fig.show()


**APP IDEAS**
- There is toggle that switches the visualizations to rollup or drilldown between visualizing by province or by region
- Disaster 1 and 2 interactions will be connected
- Disasteer 3 and 4 interactions will be connected
- Header and stories to be provided as text 